In [1]:
from __future__ import print_function
from __future__ import division

import cv2
import time
import numpy as np
import src.viztools as vv

 ### TODO
 * rgbburst style 2

In [ ]:
"""vid-viz main loop

KEYBOARD INPUTS:
    number keys - choose effect (see viztools.py for detailed keyboard input for each effect)
        0 - thresholding
        1 - alien
        3 - rgbwalk
        4 - rgbburst
    q - quit effect (then number keys are once again available for choosing a new effect)
    ` - enable toggling of border effects ('n' to toggle)
    esc - exit loop
"""

# general parameters
SOURCE = 'video'      # 'cam' | 'video' | 'image'
DISP_FULL_SCREEN = 0
ANIM_STYLE = 'loop' # 'loop' | 'pcycle' | 'rand'
                    # reverse dir eats up lots of CPU in pcycle and rand

# select video source
if SOURCE is 'cam':
    cap = cv2.VideoCapture(0)
elif SOURCE is 'video':
#     cap = cv2.VideoCapture(0)
#     cap = cv2.VideoCapture('/media/data/Dropbox/Git/vid-viz/data/tunnel_01.mp4')
#     cap = cv2.VideoCapture('/home/mattw/Videos/2016-12-18 23-15-59.mp4')
#     cap = cv2.VideoCapture('/home/mattw/Videos/2016-08-26 02-48-16.mp4')
    cap = cv2.VideoCapture('/media/data/Dropbox/Git/vid-viz/data/snowflake_02.mp4')
elif SOURCE is 'image':
    frame_orig = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/honeycomb_01.jpg')
    frame_orig = cv2.resize(frame_orig,
                            None,
                            fx=0.5,
                            fy=0.5,
                            interpolation=cv2.INTER_LINEAR)
    frame_mask = np.copy(frame_orig)

# get video properties
if SOURCE is not 'image':
    # target_fps = cap.get(cv2.CAP_PROP_FPS)
    tot_fr_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
else:
    tot_fr_count = float('inf')
    
target_fps = 20.0
ANIM_DIR = 'forward'

# initialize key handler
effect_index = None
post_process = False
key_list = [False for i in range(256)]

# initialize processing objects
effects = [vv.Threshold(),
           vv.Alien(),
           vv.RGBWalk(),
           vv.RGBBurst(),
           vv.Mask()]
num_effects = len(effects)
border = vv.Border()
# mask = effects[-1].process(frame_mask, key_list)

fr_count = 0
while(True):

    time_pre = time.time()
    
    # get frame and relevant info
    if SOURCE is not 'image':
        ret, frame = cap.read()
        # crop image to have same aspect ratio as monitor
        if SOURCE is 'cam':
            frame = frame[60:-60,:,:]
    else: #if effect_index is not num_effects-1:
        # only update frame if not updating mask
        frame = np.copy(frame_orig)

    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    key_list[key] = True
    if key == 27:
        # escape key
        break
    elif key == ord('q'):
        # quit current effect
        effect_index = None
    elif key == ord('`'):
        post_process = not post_process
        
    # update current effect
    if effect_index is None:
        for num in range(10):
            if key == ord(str(num)):
                effect_index = num
                key_list[key] = False
        
    # process frame
    if effect_index is not None and effect_index < num_effects-1:
        if post_process:
            frame = effects[effect_index].process(frame, key_list, key_lock=True)            
        else:
            frame = effects[effect_index].process(frame, key_list)

    # apply borders last to save processing time
    if post_process:
        frame = border.process(frame, key_list)
    else:
        frame = border.process(frame, key_list, key_lock=True)
        
#     if effect_index is not None and effect_index == num_effects-1:
#         mask = effects[effect_index].process(frame_mask, key_list)
#     frame = cv2.bitwise_and(frame, frame, mask=mask)

    # display frame
    if DISP_FULL_SCREEN:
        cv2.namedWindow('frame', cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.imshow('frame', frame)
    else:
        cv2.imshow('frame', frame)
        if fr_count == 0:
            cv2.moveWindow('frame', 0, 0)
    
    # control animation
    if ANIM_STYLE is 'loop':
        fr_count += 1
        if fr_count == tot_fr_count:
            # reset frame postion to 1 (not zero so window isn't moved)
            cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
            fr_count = 1
    elif ANIM_STYLE is 'pcycle':
        # update frame index
        if ANIM_DIR is 'forward':
            fr_count += 1
        elif ANIM_DIR is 'backward':
            fr_count -= 1
        # reverse direction if necessary
        if fr_count == tot_fr_count:
            fr_count -= 2
            ANIM_DIR = 'backward'
        elif fr_count == 0:
            fr_count = 1
            ANIM_DIR = 'forward'
            cap.set(cv2.CAP_PROP_POS_FRAMES, fr_count)
        # update
        if ANIM_DIR is 'backward':
            cap.set(cv2.CAP_PROP_POS_FRAMES, fr_count)

    # calculate, limit and output fps
    time_tot = time.time() - time_pre
    if time_tot < 1/target_fps:
        time.sleep(1/target_fps - time_tot)
    time_tot = time.time() - time_pre
    print('\r%03i fps' % (1.0 / time_tot), end='')
    
    # reset key list (for pressed keys that are not reset by effect object(s))
    key_list[key] = False
    
cap.release()
cv2.destroyWindow('frame')

019 fps

In [32]:
reload(vv)

<module 'src.viztools' from 'src/viztools.pyc'>

In [27]:
ord('\t')

9

In [119]:
%timeit cap.set(cv2.CAP_PROP_POS_FRAMES,1)

The slowest run took 133.82 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 183 ns per loop


In [68]:
img1 = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/test_img.png')

# img0 = 0
# img2 = cv2.addWeighted(img0, 0.1, img1, 0.9, 0)
# %timeit cv2.addWeighted(img0, 0.1, img1, 0.9, 0)

# im_width, im_height, _ = img1.shape
# %timeit cv2.warpAffine(img1, \
#             np.float32([[1, 0, 100], \
#                         [0, 1, 200]]), \
#             (im_width, im_height))

%timeit cv2.resize(img1, None, fx=2, fy=2, interpolation = cv2.INTER_LINEAR)

100 loops, best of 3: 2.51 ms per loop


### To patrol-cycle an mp4 with ffmpeg
* ffmpeg -i source.mp4 -vf reverse source_rev.mp4
* ffmpeg -f concat -i concat_list.txt -c copy concated.mp4

##### concat_list.txt
file '/path/to/vid0.mp4'

file '/path/to/vid1.mp4'